# Week 6 tutorial - Resampling methods

The materials used in this tutorial are based on the applied exercises provided in the book <font color="brown">"An Introduction to Statistical Learning with Applications in R"</font> (<a href="http://www-bcf.usc.edu/~gareth/ISL/">ISLR</a>). We are trying to demonstrate how to use R to use resampling methods and assess their performance on real-world datasets. Besides the exercises that we are going to cover in this tutorial, it is worth trying the other applied exercises by yourself in Section 5.4.

## Question 1 Predicting the default status on credit card payment

This question should be answered using the **Default** data set, which is part of the <font color="red">ISLR</font> package. This data contains the information of 10,000 customers to see how they default on their credit card debt.

In [30]:
library(ISLR)

In [31]:
head(Default)

default,student,balance,income
No,No,729.5265,44361.625
No,Yes,817.1804,12106.135
No,No,1073.5492,31767.139
No,No,529.2506,35704.494
No,No,785.6559,38463.496
No,Yes,919.5885,7491.559


This dataset consists of 10,000 observations on the following 4 variables. (The following detail is copied and pasted from <a href="https://cran.r-project.org/web/packages/ISLR/ISLR.pdf">here</a>)
* <font color="orange">default</font> A factor with levels No and Yes indicating whether the customer defaulted on their debt
* <font color="orange">student</font> A factor with levels No and Yes indicating whether the customer is a student
* <font color="orange">balance</font> The average balance that the customer has remaining on their credit card after making their monthly payment
* <font color="orange">income</font> Income of customer



We can have a look at the structure of <font color="orange">Default</font> using the <a href="https://stat.ethz.ch/R-manual/R-devel/library/utils/html/str.html">str()</a> function. The function will give us information of the data that you should know. For example, the data type of each column, the total number of observations, the total number of variables, etc. Note that the <font color="orange">default</font> and <font color="orange">student</font> are categorical variables (or factors) with two levels.

In [32]:
str(Default)

'data.frame':	10000 obs. of  4 variables:
 $ default: Factor w/ 2 levels "No","Yes": 1 1 1 1 1 1 1 1 1 1 ...
 $ student: Factor w/ 2 levels "No","Yes": 1 2 1 1 1 2 1 2 1 1 ...
 $ balance: num  730 817 1074 529 786 ...
 $ income : num  44362 12106 31767 35704 38463 ...


In [33]:
head(Default$default)

[1] No No No No No No
Levels: No Yes

We can look at the information of the variables using <a href="https://stat.ethz.ch/R-manual/R-devel/library/base/html/summary.html">summary()</a> function.

In [34]:
summary(Default)

 default    student       balance           income     
 No :9667   No :7056   Min.   :   0.0   Min.   :  772  
 Yes: 333   Yes:2944   1st Qu.: 481.7   1st Qu.:21340  
                       Median : 823.6   Median :34553  
                       Mean   : 835.4   Mean   :33517  
                       3rd Qu.:1166.3   3rd Qu.:43808  
                       Max.   :2654.3   Max.   :73554  

### 1.1 Fit a logistic regression model that uses <font color="orange">income</font> and <font color="orange">balance</font> to predict <font color="orange">default</font>. 

In [35]:
fit.glm <- glm(default ~ income + balance, data = Default, family = "binomial")
summary(fit.glm)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4725  -0.1444  -0.0574  -0.0211   3.7245  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


The predictors are significantly associated with the response (<font color="orange">default</font>) variable. We can use the Chisq test to check how well is the logistic regression model in comparison with a null model.  

In [36]:
with(fit.glm, pchisq(null.deviance - deviance, df.null - df.residual, lower.tail = FALSE))

[1] 4.540906e-292

The p-value of the Chisq test is quite low. It tells us that our model fits significantly better than an empty model.

### 1.2 Using the validation set approach, estimate the test error of this model. 

In order to do this, you must perform the following steps:

* Split the sample set into a training set and a validation set.
* Fit a multiple logistic regression model using only the training observations.
* Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the <font color="orange">default</font> category if the posterior probability is greater than 0.5.
* Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [37]:
set.seed(1234)
train <- sample(dim(Default)[1], dim(Default)[1] / 2)

In [38]:
fit.glm.t <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train)
summary(fit.glm.t)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default, subset = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0472  -0.1523  -0.0634  -0.0244   3.6569  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.111e+01  5.843e-01  -19.01  < 2e-16 ***
income       2.034e-05  6.849e-06    2.97  0.00298 ** 
balance      5.407e-03  3.056e-04   17.70  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1497.19  on 4999  degrees of freedom
Residual deviance:  839.49  on 4997  degrees of freedom
AIC: 845.49

Number of Fisher Scoring iterations: 8


In [39]:
probs <- predict(fit.glm.t, newdata = Default[-train, ], type = "response")
pred.glm <- rep("No", length(probs))
pred.glm[probs > 0.5] <- "Yes"
test.default = Default[-train,]$default
table(pred.glm, test.default)

        test.default
pred.glm   No  Yes
     No  4821  107
     Yes   18   54

The matrix shows that the fitted model predicted that a total of 4928 (4821+107) customers would not default. The sensitivity is the percentage of correct prediction of default, which is computed as 54/(54+107) = 33.54%. The specificity is the percentage of correct prediction of non-default, 4821/(4821+18) = 99.63%. Therefore, it is clear that the specifity is quite high, but the sensitivity is very low.

The validation set error, which is the fraction of the observations in the validation set that are misclassified, is calculated using the following code:

In [40]:
mean(pred.glm != test.default)

[1] 0.025

### 1.3 Repeat the process in <font color="red">1.2</font> three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained. 
In this task, we are going to predict three different splits for the observations. We used the <font color="orange">income</font> variable to split the data set. 

In [41]:
range(Default$income)
quantile(Default$income, c(0.33, 0.67))

[1]   771.9677 73554.2335

33%      67% 
25309.38 40732.33

In [42]:
train1 = (Default$income > 25306.38)
train2 = (Default$income < 25306.38 | (Default$income > 40732.33))
train3 = (Default$income < 40732.33)

In [43]:
fit.glm1 <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train1)
probs1 <- predict(fit.glm1, newdata = Default[!train1, ], type = "response")
pred.glm1 <- rep("No", length(probs1))
pred.glm1[probs1 > 0.5] <- "Yes"
test.default1 = Default[!train1,]$default
table(pred.glm1, test.default1)

         test.default1
pred.glm1   No  Yes
      No  3123   80
      Yes   42   53

In [44]:
mean(pred.glm1 != test.default1)

[1] 0.03699212

The validation error for the model is generated based on the first train data set (train1) is 3.7%.

In [45]:
fit.glm2 <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train2)
probs2 <- predict(fit.glm2, newdata = Default[!train2, ], type = "response")
pred.glm2 <- rep("No", length(probs2))
pred.glm2[probs2 > 0.5] <- "Yes"
test.default2 = Default[!train2,]$default
table(pred.glm2, test.default2)

         test.default2
pred.glm2   No  Yes
      No  3296   73
      Yes    6   27

In [46]:
mean(pred.glm2 != test.default2)

[1] 0.02322163

The validation error for the model is generated based on the second train data set (train2) is 2.32%.

In [47]:
fit.glm3 <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train3)
probs3 <- predict(fit.glm3, newdata = Default[!train3, ], type = "response")
pred.glm3 <- rep("No", length(probs3))
pred.glm3[probs3 > 0.5] <- "Yes"
test.default3 = Default[!train3,]$default
table(pred.glm3, test.default3)

         test.default3
pred.glm3   No  Yes
      No  3183   61
      Yes   17   39

In [48]:
mean(pred.glm3 != test.default3)

[1] 0.02363636

The validation error for the model is generated based on the third train data set (train3) is 2.36%. 

As can be seen, the validation error rate veries for different train data sets. It depends on which observations are included in the training set and which observations are included in the validation set.

### 1.4 Now consider a logistic regression model that predicts the probability of <font color="orange">default</font> using <font color="orange">income</font>, <font color="orange">balance</font>, and a dummy variable for <font color="orange">student</font>. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for <font color="orange">student</font> leads to a reduction in the test error rate.


In [49]:
fit.glm4 <- glm(default ~ income + balance + student, data = Default, family = "binomial", subset = train)
summary(fit.glm4)


Call:
glm(formula = default ~ income + balance + student, family = "binomial", 
    data = Default, subset = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0769  -0.1527  -0.0620  -0.0238   3.6193  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.058e+01  6.653e-01 -15.908   <2e-16 ***
income       6.690e-06  1.104e-05   0.606    0.544    
balance      5.472e-03  3.102e-04  17.640   <2e-16 ***
studentYes  -5.027e-01  3.175e-01  -1.583    0.113    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1497.2  on 4999  degrees of freedom
Residual deviance:  837.0  on 4996  degrees of freedom
AIC: 845

Number of Fisher Scoring iterations: 8


In the new model, there are some variables which are not significantly associated with the response variable. 

In [50]:
probs4 <- predict(fit.glm4, newdata = Default[-train, ], type = "response")
pred.glm4 <- rep("No", length(probs4))
pred.glm4[probs4 > 0.5] <- "Yes"
test.default = Default[-train,]$default
table(pred.glm4, test.default)

         test.default
pred.glm4   No  Yes
      No  4820  108
      Yes   19   53

In [51]:
mean(pred.glm4 != test.default)

[1] 0.0254

The validation error has increased slightly in comparison with the model without dummy variable for <font color="orange">student</font>. It seems adding <font color="orange">student</font> variable does not lead to reduction in validation error.

## Question 2 Using Bootstrap on predicting the default status

We continue to consider the use of a logistic regression model to predict the probability of <font color="orange">default</font> using <font color="orange">income</font> and <font color="orange">balance</font> on the <font color="orange">Default</font> data set. In particular, we will now compute estimates for the standard errors of the <font color="orange">income</font> and <font color="orange">balance</font> logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the <font color="blue">glm()</font> function. Do not forget to set a random seed before beginning your analysis.

### 2.1 Using the <font color="blue">summary()</font> and <font color="blue">glm()</font> functions, determine the estimated standard errors for the coefficients associated with <font color="orange">income</font> and <font color="orange">balance</font> in a multiple logistic regression model that uses both predictors.

In [52]:
library(ISLR)

In [53]:
fit.glm <- glm(default ~ income + balance, data = Default, family = "binomial")
summary(fit.glm)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4725  -0.1444  -0.0574  -0.0211   3.7245  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


The <font color="blue">glm()</font> estimates the standard errors for the coefficients β0, β1 and β2 are respectively $0.4347564$, $4.9851672x10^{-6}$ and $2.2737314x10^{-4}$.
Large standard errors for the logistic regression coefficients may lead to invalid statistical inferences. 

### 2.2 Write a function, <font color="blue">boot.fn()</font>, that takes as input the <font color="orange">Default</font> data set as well as an index of the observations, and that outputs the coefficient estimates for <font color="orange">income</font> and <font color="orange">balance</font> in the multiple logistic regression model.

In [54]:
boot.fn <- function(data, index) {
    fit <- glm(default ~ income + balance, data = data, family = "binomial", subset = index)
    return (coef(fit))
}

In [55]:
index <- sample(dim(Default)[1], dim(Default)[1] / 2)
boot.fn(Default, index)

(Intercept)        income       balance 
-1.139212e+01  2.353168e-05  5.466861e-03

This function can make the logistic model based on sample data sets.

In [56]:
boot.fn(Default, 1000)

(Intercept)      income     balance 
  -22.56607          NA          NA

### 2.3 Use the <a href="https://cran.r-project.org/web/packages/boot/boot.pdf">boot()</a> function together with your <font color="blue">boot.fn()</font> function to estimate the standard errors of the logistic regression coefficients for <font color="orange">income</font> and <font color="orange">balance</font>.
 

In [57]:
library(boot)
set.seed(1234)
boot(Default, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Default, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
         original        bias     std. error
t1* -1.154047e+01 -1.909078e-02 4.309840e-01
t2*  2.080898e-05 -2.601640e-08 4.787041e-06
t3*  5.647103e-03  1.128372e-05 2.232473e-04

The bootstrap estimates of the standard errors for the coefficients β0, β1 and β2 are respectively $0.43$, $4.787 x 10^{-6}$ and $2.232 x 10^{-4}$.

### 2.4 Comment on the estimated standard errors obtained using the <font color="blue">glm()</font> function and using your bootstrap function.

The standard errors for coefficients of both functions are pretty close to each other. However, the standard errors obtained from bootstrap function is slightly lower.

In [ ]:
set.seed(1234)
boot(Default, boot.fn, 10000)